In [1]:
import os
import src.data_manager as data_manager
import src.gfdl as gfdl
import src.util as util
import src.datelabel as datelabel
import src.shared_diagnostic as shared_diagnostic
import yaml

In [2]:
default_args = util.read_yaml('./src/config.yml')
cmdline_args = {'CODE_ROOT':os.getcwd()}
test_dir = '/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp'
test_case = {
    'CASENAME': 'CM4_historical', 'model': 'CM4', 
     'variable_convention': 'CMIP',
    'FIRSTYR': 1970, 'LASTYR': 1989,
    'pod_list': []
}
test_varlist = [
    {'var_name':'rlut', 'freq':'day'},
    {'var_name':'pr', 'freq':'day'},
    {'var_name':'omega500', 'freq':'day'},
    {'var_name':'u200', 'freq':'day'},
    {'var_name':'u850', 'freq':'day'}
]

In [ ]:
reload(data_manager)
reload(gfdl)

In [13]:
reload(gfdl)

<module 'src.gfdl' from 'src/gfdl.py'>

In [14]:
config = util.parse_mdtf_args(None, cmdline_args, default_args)
util.PathManager(config['paths'])
dm = gfdl.GfdlppDataManager(test_dir, test_case)
pod = shared_diagnostic.Diagnostic('MJO_prop_amp')
dm.pods = [pod]
for pod in dm.pods:
    dm._setup_pod(pod)
ds = dm.pods[0].varlist[0].copy()
print ds

DataSet({'CF_name': 'pr_var', 'requirement': 'required', 'name': 'pr_var', 'date_freq': DateFrequency('1da'), 'required': True, 'date_range': DateRange('1970-1989'), 'remote_resource': None, 'var_name': 'pr_var', 'name_in_model': 'pr', 'units': None, 'freq': 'day', 'local_resource': '/local2/home/MDTF/inputdata/model/CM4_historical/day/CM4_historical.pr.day.nc', 'alternates': []})


In [ ]:
dm.query_dataset(ds)
print ds

In [4]:
dm._query_dataset_and_alts(ds)

[DataSet({'CF_name': 'pr_var', 'requirement': 'required', 'name': 'pr_var', 'date_freq': DateFrequency('1da'), 'required': True, 'date_range': DateRange('1970-1989'), 'remote_resource': [DataSet({'end_date': '19741231', 'date_freq': DateFrequency('1da'), 'component': 'atmos_cmip_2deg_daily_2D', 'date_range': DateRange('19700101-19741231'), 'remote_resource': 'atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19700101-19741231.pr.nc', 'file': 'atmos_cmip_2deg_daily_2D.19700101-19741231.pr.nc', 'field_name': 'pr', 'local_resource': None, 'name': None, 'chunk_freq': DateFrequency('5yr'), 'units': None, 'start_date': '19700101', 'dir': 'atmos_cmip_2deg_daily_2D/ts/daily/5yr'}), DataSet({'end_date': '19791231', 'date_freq': DateFrequency('1da'), 'component': 'atmos_cmip_2deg_daily_2D', 'date_range': DateRange('19750101-19791231'), 'remote_resource': 'atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19750101-19791231.pr.nc', 'file': 'atmos_cmip_2deg_daily_2D.197501

In [ ]:
for f in ds.remote_resource:
    print f.remote_resource
print len(ds.remote_resource)
print ds.remote_resource[0]

In [ ]:
cmpts = dm._select_model_component([ds])
print cmpts

In [16]:
for pod in dm.pods:
    new_varlist = []
    for var in pod.varlist:
        try:
            new_varlist.extend(dm._query_dataset_and_alts(var))
        except DataQueryFailure:
            print "Data query failed on pod {}".format(pod.name)
            raise
    pod.varlist = new_varlist

In [17]:
files = dm.plan_data_fetching()
for f in files:
    print f.remote_resource

atmos_cmip_2deg_daily_3D/ts/daily/5yr/atmos_cmip_2deg_daily_3D.19800101-19841231.hus.nc
atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19800101-19841231.prw.nc
atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19700101-19741231.prw.nc
atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19800101-19841231.pr.nc
atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19750101-19791231.pr.nc
atmos_cmip_2deg_daily_3D/ts/daily/5yr/atmos_cmip_2deg_daily_3D.19750101-19791231.hus.nc
atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19850101-19891231.prw.nc
atmos_cmip_2deg_daily_3D/ts/daily/5yr/atmos_cmip_2deg_daily_3D.19700101-19741231.hus.nc
atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19750101-19791231.prw.nc
atmos_cmip_2deg_daily_2D/ts/daily/5yr/atmos_cmip_2deg_daily_2D.19700101-19741231.pr.nc
atmos_cmip_2deg_daily_3D/ts/daily/5yr/atmos_cmip_2deg_daily_3D.19850101-19891231.hus.nc
atmos_cmip_2deg_daily_2D/ts/daily/5

In [ ]:
foo = [{'a':[1,2], 'b':[5]}, {'a':[7,8]}]
print [d['a'] for d in foo]
print list(*(d['a'] for d in foo))

In [ ]:
foo = [{'a':[1,2], 'b':[5]}, {'a':[7,8]}]
def iter_foo():
    for d in foo: yield d
def iter_foo2():
    for d in foo: yield d['a'][0]
for d in iter_foo():
    d['c'] = [69]
print foo

In [ ]:
print set(iter_foo2())
# print len(iter_foo())
for i,d in enumerate(iter_foo()):
    print i, d['a']
print itertools.count(iter_foo())

In [ ]:
goo = (d['a'] for d in foo)
for g in goo:
    print g

In [ ]:
foo = [[],[1,2],[],[3,4]]
print list(itertools.ifilter(None, foo))
print filter(None, foo)

In [ ]:
print files[0].remote_resource[0].file
print files[0].remote_resource[1].file

In [ ]:
files_to_fetch = []
ds_list = [d for d in ds.remote_resource if (d.component in cmpts)]
# take longest chunk frequency (revisit?)
chunk_freq = max({d.chunk_freq for d in ds_list})
ds.remote_resource = [d for d in ds_list if (d.chunk_freq == chunk_freq)]
assert ds.remote_resource # shouldn't have eliminated everything
files_to_fetch.extend(ds.remote_resource)
print files_to_fetch
for f in files_to_fetch:
    f.local_resource = dm.local_path(f)
foo2 = [f for f in set(files_to_fetch) if not dm.local_data_is_current(f)]
print foo2

## junk below here

In [ ]:
util.find_files(test_dir, '*/ts/daily/*.rlut.nc')

In [ ]:
for f in files_to_fetch:
    print f.remote_resource

In [ ]:
import tempfile
import os

In [ ]:
tempfile.tempdir = os.getcwd()
foo1 = tempfile.mkdtemp(prefix='MDTF_temp_')
print foo1
os.rmdir(foo1)

In [ ]:
print tempfile.tempdir

In [ ]:
print ds.__hash__()
print hex(ds.__hash__())

In [ ]:
print os.listdir(test_dir)

In [ ]:
def test_find(root_dir, dataset):
    cmpts = [d for d in os.listdir(root_dir) if not d.startswith('.')]
    candidates = []
    query = '*.{}.nc'.format(dataset.name_in_model)
    for rp1 in cmpts:
        rp2 = os.path.join(rp1, 'ts', dataset.date_freq.format_frepp())
        ap2 = os.path.join(root_dir, rp2)
        if os.path.exists(ap2):
            chunk_freqs = [d for d in os.listdir(ap2) if not d.startswith('.')]
            for rp3 in chunk_freqs:
                ap3 = os.path.join(ap2, rp3)
                print ap3
                paths = util.run_command([
                    'find', ap3, '-name', query, '-print','-quit'
                ])
                if paths:
                    print 'GOOD'
                    candidates.append(ap3)
    return candidates

In [ ]:
test_find(test_dir, ds)

In [ ]:
os.listdir('/archive/oar.gfdl.cmip6/CM4/warsaw_201710_om4_v1.0.1/CM4_historical/gfdl.ncrc4-intel16-prod-openmp/pp/atmos_cmip_2deg_daily_2D/ts/daily/35yr')


In [ ]:
    def query_dataset2(dataset, root_dir):
        """Return set of candidate directories for data files.
        """
        if 'component' in dataset:
            cmpts = [dataset.component]
        else:
            cmpts = [d for d in os.listdir(root_dir) if not d.startswith('.')]
        candidate_dirs = []
        suffix_query = '.{}.nc'.format(dataset.name_in_model)
        for component in cmpts:
            subdir_rel = os.path.join(component, 'ts', dataset.date_freq.format_frepp())
            subdir_abs = os.path.join(root_dir, subdir_rel)
            if not os.path.exists(subdir_abs):
                continue
            chunk_freqs = [d for d in os.listdir(subdir_abs) if not d.startswith('.')]
            for freq in chunk_freqs:
                paths = util.run_command([
                    'find', os.path.join(subdir_abs, freq), '-name', '*'+suffix_query, '-print', '-quit'
                ])
                if paths:
                    candidate_dirs.append(os.path.join(subdir_rel, freq))
        if not candidate_dirs:
            raise DataQueryFailure(dataset, 'No files found in {}'.format(root_dir))

        for d in candidate_dirs:
            files = [dm.parse_pp_path(f) for f in os.listdir(os.path.join(root_dir,d)) \
                if f.endswith(suffix_query)]
            try:
                remote_range = datelabel.DateRange([ds.date_range for ds in files])
            except ValueError:
                # Something's messed up with remote files if we get here
                # should probably log an error
                continue
            if remote_range.contains(dataset.date_range):
                dataset.remote_resource.extend(
                    [ds for ds in files if (ds.date_range in dataset.date_range)]
                )
        if not dataset.remote_resource:
            raise DataQueryFailure(dataset, 
                "Couldn't cover date range {} with files in {}".format(
                    dataset.date_range, root_dir))

In [ ]:
query_dataset2(ds, test_dir)